In [4]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [5]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "  "
    consumer_secret = "  "
    access_token = "  "
    access_token_secret = "  "

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -123.90990783548145, 46.31045694224361, -117.10895501117177, 48.980679102707946,  # Washington
                 -74.80392586084015, 40.2988386351859, -72.26644814654395, 41.00668669952152,  # New York
                 -124.08044706635084, 41.88517481753033, -114.8415039347023, 32.96029841383517, # California
                ]  

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['Geography, GIS, Data Science'], is_async=True)

{'created_at': 'Wed Jan 27 23:44:54 +0000 2021', 'id': 1354576144641757184, 'id_str': '1354576144641757184', 'text': "RT @GidMK: My new piece is on Australia, New Zealand, COVID-19, and why it's nonsense to say that it's just luck (or geography) that has pu…", 'source': '<a href="https://tapbots.com/software/tweetbot/mac" rel="nofollow">Tweetbot for Mac</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 352649144, 'id_str': '352649144', 'name': 'Joshua Carver', 'screen_name': 'joe_carver', 'location': 'Norwich', 'url': 'http://vimeo.com/joshuacarver', 'description': 'Filmmaker. Photographer. Retweeter', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 202, 'friends_count': 145, 'listed_count': 16, 'favourites_count': 1593, 'statuses_count': 9169, 'created_at': 'Wed Aug 10 22:29:07 +0000 2011', 'utc_off